![Nuclio logo](https://nuclio.school/wp-content/uploads/2018/12/nucleoDS-newBlack.png)

# Reinforced Learning con Q-learning - El taxi autónomo

El primer ejemplo de reinforced learning más en serio que lo del multi-armed bandit

## 1. Instalemos las librerias en nuestro entorno local para tener gym y poder mostrar los videos grabados

Para hacer funcionar el proyecto hemos de instalar gym:

<code>pip install gym</code>


In [ ]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) # error only
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

Las dos siguientes funciones son para poder grabar en video los entornos de gym y mostrarlos

Para activar el video, solo teneis que hacer "env = wrap_env(env)"

In [ ]:
def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

## 2. A por el entorno del Taxi

In [ ]:
entorno = gym.make("Taxi-v3").env

entorno.render()

In [ ]:
entorno.reset()
entorno.render()

Esto se parece un poco a lo que hemos estado viendo en las slides, la única diferencia (que era así en la version v2 de Taxi) es que el taxi esta en la posición R, en vez de la (1,3) como teniamos pintado antes

El universo de la interfaz gym es <code> entorno </code>. Tenemos algunos métodos que iremos usando que os pueden ser útiles:

<ul>
  <li><code> entorno.reset </code> Pone el entorno en posición de fábrica, y nos devuelve un estado inicial random</li>
  <li><code> entorno.step(action) </code> Paso del entorno tras un incremento de tiempo. Esto nos devuelve:
  <ul>
    <li><b>observation</b>: Observaciones del entorno</li>
    <li><b>reward</b>: La recompensa que recogemos</li>
    <li><b>done</b>: indica si hemos recogido o dejado un pasajero de forma exitosa, lo que denominaremos un <i>episodio</i></li>
    <li><b>info</b>: Informacion adicional como la performance o la latencia de cara al debug</li>
  </ul>
  <li><code> entorno.render </code> Pinta un frame del entorno (muy útil para hacernos una idea del mismo)

Nota adicional: hemos hecho <code>gym.make("Taxi-v3").<b>env</b></code> para evitar que nos pare a las 200 iteraciones que es como funciona por defecto Gym





### Hagamos memoria del problema

Tenemos 4 localizaciones (con diferentes letras), y nuestro trabajo es recoger un pasajero en una de ellas y llevarlo a otra. Recibimos 20 puntos por una entrega exitosa, y perdemos 1 punto por cada paso de tiempo (para optimizar el recorrido). Hemos incorporado una penalización de 10 puntos por entregas o recogidas ilegales en localizaciones equivocadas.

In [ ]:
entorno.reset()
entorno.render()

Fijaros que ahora el taxi esta en otra localización.

Más cosas del entorno:
*   La cajita amarilla es el taxi
*   Las paredes o setos por donde no hay carretera son los pipes (|)
*   R, G, Y, B son los puntos de recogida y entrega. El color <font color="blue"><b>azul</b></font> indica punto de recogida, el color <font color="purple"><b>purpura</b></font> indica punto de entrega.

In [ ]:
print("Espacio acción {}".format(entorno.action_space))
print("Espacio estado {}".format(entorno.observation_space))


Fijaros que el Espacio Acción (action space) es de tamaño 6 y el Espacio Estado (state space) tiene tamaño 500. 

En terminos de mapeado que sepais que las acciones van de 0 a 5 con estos valores:

*   0 = sur
*   1 = norte
*   2 = este
*   3 = oeste
*   4 = recoger pasajero
*   5 = dejar pasajero

Indice de puntos de recogida o entrega

*   0 = R (0,0)
*   1 = G (0,4)
*   2 = Y (4,0)
*   3 = B (4,3)

Pero antes que nada, vamos a dar un vistazo a esa posición (3,1) que habiamos visto, veremos la información de ese estado. Con un pasajero esperando en la posición 2 (<font color="blue"><b>Y</b></font>) con destino 0 (<font color="purple"><b>R</b></font>)


In [ ]:
estado = entorno.encode(3, 1, 2, 0) # (fila taxi, columa taxi, indice posicion pasajero, indice posición destino)
print("Estado:", estado)

entorno.s = estado # Podemos asignar el estado actual al que queramos definir
entorno.render()

### La tabla de recompensas

Com podeis ver de las 0 a la 499 coordenadas que tenemos en nuestro entorno, seria interesante si lo que hemos dispuesto en nuestras recompensas en la presentación es lo mismo que hay en Gym

In [ ]:
entorno.P[328]

Los índices del diccionario de la posición 328 son las acciones a tomar desde es punto. Y siguen una estructura muy clara:

<code>{acción: [(probabilidad, siguiente estado, recompensa, realizado)]}</code>

Como podéis ver, en este entorno hemos asignado una <code>probabilidad</code> a cada acción del 100% (no hacemos distinciones, ni forzamos un comportamiento del agente.

<code>siguiente estado</code> nos indica en que coordenadas terminariamos si tomaramos la acción del índice.

La <code>recompensa</code> se muestra en esa tercera posición, con los valores de -1 si "añadimos" un paso de tiempo más y -10 si al taxi se le ocurre recoger o dejar a un pasajero. Y si miraramos en la coordenada de destino correcta con un pasajero dentro del taxi, aparecería un bonito 20 en <code>recompensa</code> en la acción 5 (dejar pasajero).

La posición <code>realizado</code> se usa para decirnos que hemos dejado un pasajero en la localización correcta. Cada entrega existosa sera nuestro final de **episodio**.

## 3. Que pasa si lo dejamos hacer solo al taxi (no Reinforcement Learning)

Por el método de la fuerza bruta. Se trata de usar la tabla <code>P</code> de recompensas, que nos será la guia para la navegación del taxi.

La idea es montar un loop infinito que no se parará hasta que el taxi deje a un pasajero en su destino (un simple **episodio**). O cuando recibamos una recompensa de 20. 

El método <code>entorno.action_space.sample()</code> toma una acción de forma aleatoria de todas las posibles acciones.

Veamos que ocurre....

In [ ]:
entorno.s = 328 

epochs = 0
castigo, recompensa = 0, 0

total_epochs = 0
total_castigos = 0

frames = [] # para la animación!!

finepisodio = False

while not finepisodio:
    accion = entorno.action_space.sample()
    estado, recompensa, finepisodio, info = entorno.step(accion)

    if recompensa == -10:
        castigo += 1
    
    # Ponemos cada frame renderizado dentro de un diccionatio para la animación
    frames.append({
        'frame': entorno.render(mode='ansi'),
        'episodio': 0,
        'state': estado,
        'action': accion,
        'reward': recompensa
        }
    )

    epochs += 1

total_castigos += castigo
total_epochs += epochs
episodios = 1
    
print("Pasos de tiempo usados: {}".format(epochs))
print("Penalizaciones acumuladas: {}".format(castigo))

tiempomedio_random = total_epochs / episodios
castigomedio_random = total_castigos / episodios

print(f"Resultados después de {episodios} episodios:")
print(f"Media de tiempo por episodio: {tiempomedio_random}")
print(f"Media de castigos por episodio: {castigomedio_random}")



### Pintemos los resultados en una bonita animación

In [ ]:
from IPython.display import clear_output
from time import sleep

def pintar_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Episodio: {frame['episodio']}")
        print(f"Tiempo: {i + 1}")
        print(f"Estado: {frame['state']}")
        print(f"Accion: {frame['action']}")
        print(f"Recompensa: {frame['reward']}")
        sleep(.1)

pintar_frames(frames)

Le ha costado la vida entregar al pasajero a su destino... no está mal.

## 4. El turno de Q-Learning

Recordemos que ahora hay que actualizar los valores de Q(a,s) en la Q-table, porque es de esa guía de donde sacaremos las mejores acciones para nuestro agente, el taxi.

Valores más altos o mejores de Q indicaran para donde hemos de ir con nuestro taxi. Es decir, si el taxi se encuentra en un estado donde le espera un pasajero, es bastane probable que los valores para <code>recoger pasajero</code> sean mayores que el resto de acciones disponibles.


### 4.1 Inicializando la Q-table a una matriz de 500x6 llena de ceros

In [ ]:
import numpy as np
q_table = np.zeros([entorno.observation_space.n, entorno.action_space.n])

print(q_table.shape)

Fijamos el tiempo, y nos cargamos unas cuantas librerias necesarias

In [ ]:
%%time

from IPython.display import clear_output

### 4.2 Montemos el código para el entrenamiento

Definiremos los parámetros de actualización de la tabla Q (alpha, gamma) y el epsilon... ¡¡si nuestro amigo epsilon codicioso!!



In [ ]:
alpha = 0.1 # Nuestro learning rate
gamma = 0.9 # Nuestro descuento a la recompensa
epsilon = 0.1 # Epsilon codicioso!!

Definimos una variables para poder pintar metricas


In [ ]:
all_epochs = []
all_penalties = []

Montamos el bucle de episodios, donde dentro meteremos el bucle del episodio en sí.

In [ ]:
for i in range(1, 100001):
  estado = entorno.reset() # Arrancamos el entorno en una posición aleatoria cada vez

  epochs, castigo, recompensa = 0, 0, 0
 
  finepisodio = False

  while not finepisodio:
    # Aquí va nuestro amigo Epsilon greedy, para fijar cuando explorarmos o explotamos
    if np.random.random() < epsilon:
      accion = entorno.action_space.sample() # Exploramos el espacio de acciones
    else: 
      accion = np.argmax(q_table[estado]) # Explotamos lo aprendido
    
    siguiente_estado, recompensa, finepisodio, info = entorno.step(accion)

    # Actualicemos los valores de la tabla Q en la posicion accion, estado tras ver lo que nos ha pasado
    # Almacenamos el valor anterior de la tabla Q
    valor_anterior = q_table[estado, accion]

    # De la posicion estados, me quedo con el valor de la accion que daria el valor más alto en la tabla Q
    siguiente_max = np.max(q_table[siguiente_estado])

    # Calculamos la formulita de actualización de Q(a,s)
    nuevo_valor = (1 - alpha) * valor_anterior + alpha * (recompensa + gamma * siguiente_max)
    
    q_table[estado, accion] = nuevo_valor

    if recompensa == -10:
      castigo += 1

    estado = siguiente_estado
    epochs += 1

  if i % 100 == 0:
    clear_output(wait=True)
    print(f"Episodio: {i}")



print("Entrenamiento terminado.\n")


In [ ]:
q_table[328]

Una vez terminado el entrenamiento, veamos los valores de la posición 328 para cada acción disponible en ese entorno según lo aprendido por el taxi.

In [ ]:
q_table[328]

La acción 1 (ir al norte) es la que tiene mejor "recompensa"

### 4.3 Evaluemos que tal lo hace nuestro agente

Ahora ya no exploramos más, así que la parte de Epsilon Greedy la quitamos y tiramos directamente de los valores de Q-table

In [ ]:
total_epochs, total_castigos = 0, 0
episodios = 100

frames = [] # para la animación!!

for i in range(episodios):
    estado = entorno.reset()
    epochs, castigo, recompensa = 0, 0, 0
    
    finepisodio = False
    
    while not finepisodio:
        accion = np.argmax(q_table[estado])
        estado, recompensa, finepisodio, info = entorno.step(accion)

        if recompensa == -10:
          castigo += 1
        
        epochs += 1
         

        # Ponemos cada frame renderizado dentro de un diccionatio para la animación
        frames.append({
          'frame': entorno.render(mode='ansi'),
          'episodio': i,
          'state': estado,
          'action': accion,
          'reward': recompensa
          }
        )

    total_castigos += castigo
    total_epochs += epochs



tiempomedio_qlearning = total_epochs / episodios
castigomedio_qlearning = total_castigos / episodios

print(f"Resultados después de {episodios} episodios:")
print(f"Media de tiempo por episodio: {tiempomedio_qlearning}")
print(f"Media de castigos por episodio: {castigomedio_qlearning}")

In [ ]:
pintar_frames(frames)